# Practica 0
## Diego Rodríguez y Alejandro Jiménez
## Grupo 1463

In [1]:
from Datos import Datos

if __name__ == '__main__':
    d1 = Datos("./ConjuntoDatos/german.data")
    print(d1.datos)
    d2 = Datos("./ConjuntoDatos/tic-tac-toe.data")
    print(d2.datos)

[[ 0.  6.  8. ... 51. 52.  1.]
 [ 1. 48.  6. ... 50. 52.  2.]
 [ 3. 12.  8. ... 50. 52.  1.]
 ...
 [ 3. 12.  6. ... 50. 52.  1.]
 [ 0. 45.  6. ... 51. 52.  2.]
 [ 1. 45.  8. ... 50. 52.  1.]]
[[ 2.  5.  8. ... 22. 25. 28.]
 [ 2.  5.  8. ... 23. 25. 28.]
 [ 2.  5.  8. ... 22. 26. 28.]
 ...
 [ 1.  5.  7. ... 22. 26. 27.]
 [ 1.  5.  7. ... 22. 26. 27.]
 [ 1.  4.  8. ... 23. 26. 27.]]


## Memoria

Lo que más nos ha costado en la práctica ha sido retomar python y comenzar a usar pandas y numpy. Nos ha llevado tiempo de investigación averiguar como utilizar los arrays de numpy y que son los dataframes de pandas, pero una vez sabíamos utilizarlos han resultado muy útiles en la práctica.

También nos han surgido varias dudas de implementación durante el desarrollo:

1. No sabiámos bien si necesitabamos crear un único diccionario para las variables nominales or un diccionario distimto para campo nominal. Hemos decidido implementar un diccionario de diccionarios donde la clave de los diccionarios de primer orden es el nombre de la columna y su valor es el diccionario que le corresponde a los valores nominales de la columna. Si esta implementaciçon resultase poco eficiente, en un futuro se podría sustituir por una lista de diccionarios donde los diccionarios estén indexados por número de fila teniendo en cuenta el array AtributosNominales.

2. No comprendíamos tampoco si se debía sustituir directamente los valores en el array de datos. Nuestra decisión de implementación ha sido utilizar un numpy array temporal y luego pasar los datos al array de la clase llamado Datos teniendo en cuanta los valores del diccionario. Lo que nos resultaba un poco contra intuitivo era que si quería volver a recuperar en el diccionario las claves, estaría buscando por valor en vez de por clave, que es como generalmente se utiliza un diccionario.

### Trabajo en pareja

Para organizarnos hemos decidido trabajar desde VSC utilizando la extensión live share para poder hacer modificaciones simultáneas y en tiempo real en el mismo archivo. Esta extensión también tiene la habilidad de comunicarse por audio así que crea un entorno de trabajo en pareja agradable y eficaz que mantendremos para las futurras prácticas.